# Import packages and initialize variables

In [3]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from mistralai.models.jobs import TrainingParameters
import json
import pandas as pd

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)
model= 'mistral-small-latest'

### Test Mistral API

In [36]:
messages = [
    ChatMessage(role="user", 
                content="Qu'est-ce que le machine learning ?")
]

response = client.chat(
    model=model,
    messages=messages
)

chat_response = response.dict()['choices'][0]['message']['content']

In [37]:
chat_response

"Le machine learning, ou apprentissage automatique en français, est une branche de l'intelligence artificielle qui permet aux ordinateurs et aux machines d'apprendre à partir de données et d'améliorer leurs performances sur une tâche spécifique sans être explicitement programmés.\n\nIl existe plusieurs types de machine learning, notamment l'apprentissage supervisé, l'apprentissage non supervisé et l'apprentissage par renforcement. L'apprentissage supervisé consiste à entraîner un modèle à partir de données étiquetées, c'est-à-dire de données pour lesquelles la réponse correcte est déjà connue. L'apprentissage non supervisé, quant à lui, consiste à entraîner un modèle à partir de données non étiquetées, c'est-à-dire de données pour lesquelles la réponse correcte n'est pas connue. L'apprentissage par renforcement consiste à entraîner un modèle à partir d'une récompense ou d'une pénalité reçue en fonction de ses actions.\n\nLe machine learning a de nombreuses applications, notamment dans 

## Conversion of the scrapped data into a jsonl file for the Mistral API Fine Tuning

In [4]:
df1 = pd.read_csv('stackoverflow1.csv').dropna()
df2 = pd.read_csv('stackoverflow2.csv').dropna()
df3 = pd.read_csv('stackoverflow3.csv').dropna()

In [5]:
df = pd.concat([df1, df2, df3])

In [6]:
def replace_str(_df: pd.DataFrame, _column: str):
    _df[_column] = _df[_column].str.replace('\n', ' ')
    _df[_column] = _df[_column].str.replace('\r', ' ')
    _df[_column] = _df[_column].str.replace('\t', ' ')
    _df[_column] = _df[_column].str.lower()\
        .str.replace(r'[^\w\s/<>=]', '')\
        .str.replace(r'\s+', ' ')
    return _df

In [7]:
def pre_process_df(_df: pd.DataFrame):
    _df = _df.dropna()
    _df.drop_duplicates(inplace=True)
    _df.reset_index(inplace=True, drop=True)
    _df['answer'] = _df['answer'].str.split('\nShare').str[0]
    _df = replace_str(_df, 'answer')
    _df = replace_str(_df, 'question')
    return _df

In [8]:
df = pre_process_df(df)

In [9]:
def count_tokens(text):
    if pd.isna(text):
        return 0
    return len(text.split())

token_counts = df.applymap(count_tokens)

total_token_count = token_counts.sum().sum()

print(f'Total number of tokens in the DataFrame: {total_token_count}')

Total number of tokens in the DataFrame: 3706313


/var/folders/vf/qcsfk0590zj_zrqnrs3xwnxr0000gn/T/ipykernel_30125/2006679773.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  token_counts = df.applymap(count_tokens)


In [10]:
data_list = []
for index, row in df.iterrows():
    data_list.append({
        "rôle": 'user',
        "output": row['question']
    })
    data_list.append({
        "rôle": 'assistant',
        "output": row['answer']
    })

In [11]:
df.to_parquet('stackoverflow.parquet')

### If you have parquet file, run the first cell and here

In [12]:
df = pd.read_parquet('stackoverflow.parquet')

df_train = df.sample(frac=0.9)
df_eval = df.drop(df_train.index)

In [13]:
jsonl_path = 'training_file.jsonl'

In [14]:
records = df_train.to_dict(orient='records')

with open(jsonl_path, 'w') as file:
    for index, row in enumerate(records):
        user_message = {
            "role": "user",
            "content": row['question']
        }
        assistant_message = {
            "role": "assistant",
            "content": row['answer']
        }
        jsonl_format = {"messages": [user_message, assistant_message]}
        file.write(json.dumps(jsonl_format) + '\n')

print(f'JSONL file saved at {jsonl_path}')

JSONL file saved at training_file.jsonl


In [15]:
jsonl_path = 'evaluation_file.jsonl'

In [16]:
records = df_eval.to_dict(orient='records')

with open(jsonl_path, 'w') as file:
    for index, row in enumerate(records):
        user_message = {
            "role": "user",
            "content": row['question']
        }
        assistant_message = {
            "role": "assistant",
            "content": row['answer']
        }
        jsonl_format = {"messages": [user_message, assistant_message]}
        file.write(json.dumps(jsonl_format) + '\n')

print(f'JSONL file saved at {jsonl_path}')

JSONL file saved at evaluation_file.jsonl


## Upload to Mistral API of the training data and evaluation data

In [17]:
with open("training_file.jsonl", "rb") as f:
    training_data = client.files.create(file=("training_file.jsonl", f)) 

print("Training file uploaded")

with open("evaluation_file.jsonl", "rb") as f:
    evaluation_data = client.files.create(file=("evaluation_file.jsonl", f)) 

print("Evaluation file uploaded")

Training file uploaded
Evaluation file uploaded


## Fine Tuning of the model

In [19]:
created_jobs = client.jobs.create(
        model=model,
        training_files=[training_data.id],
        validation_files=[evaluation_data.id],
        hyperparameters=TrainingParameters(
        training_steps=50,
        learning_rate=0.0001,
        )
)
created_jobs

Job(id='11adbf10-bc96-4f91-bd95-b7ff41f2b83c', hyperparameters=TrainingParameters(training_steps=50, learning_rate=0.0001), fine_tuned_model=None, model='mistral-small-latest', status='QUEUED', job_type='FT', created_at=1719587199, modified_at=1719587199, training_files=['a1818d5d-1aa9-47ec-af49-4de158e3c7d8'], validation_files=['f73524a6-0533-4f91-8a13-b3bef2a83db2'], object='job', integrations=[])

In [23]:
retrieved_jobs = client.jobs.retrieve(created_jobs.id)

print(retrieved_jobs)

id='11adbf10-bc96-4f91-bd95-b7ff41f2b83c' hyperparameters=TrainingParameters(training_steps=50, learning_rate=0.0001) fine_tuned_model=None model='mistral-small-latest' status='RUNNING' job_type='FT' created_at=1719587199 modified_at=1719587199 training_files=['a1818d5d-1aa9-47ec-af49-4de158e3c7d8'] validation_files=['f73524a6-0533-4f91-8a13-b3bef2a83db2'] object='job' integrations=[] events=[Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719587199), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719587199)] checkpoints=[] estimated_start_time=None


## Le Scrapper model 

In [21]:
retrieved_jobs

DetailedJob(id='11adbf10-bc96-4f91-bd95-b7ff41f2b83c', hyperparameters=TrainingParameters(training_steps=50, learning_rate=0.0001), fine_tuned_model=None, model='mistral-small-latest', status='RUNNING', job_type='FT', created_at=1719587199, modified_at=1719587199, training_files=['a1818d5d-1aa9-47ec-af49-4de158e3c7d8'], validation_files=['f73524a6-0533-4f91-8a13-b3bef2a83db2'], object='job', integrations=[], events=[Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719587199), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719587199)], checkpoints=[], estimated_start_time=None)

In [25]:
chat_response = client.chat(
    model=retrieved_jobs.model,
    messages=[ChatMessage(role='user', content="Qu'est-ce que le scrapping ?")]
)

In [26]:
chat_response.dict()['choices'][0]['message']['content']

"Le scrapping, également connu sous le nom de web scrapping ou data scraping, est une technique utilisée pour extraire des données de sites web. Il s'agit d'un processus automatisé qui permet de collecter et d'analyser des informations en ligne. Les robots d'exploration de données, ou scrapers, sont utilisés pour naviguer sur les sites web et extraire les données spécifiques nécessaires, telles que des prix, des avis, des coordonnées, etc. Le scrapping peut être utile pour des recherches, des analyses de marché, la veille concurrentielle, ou encore pour alimenter une base de données. Cependant, il est important de noter que le scrapping doit être effectué dans le respect des conditions d'utilisation des sites web et des lois sur la protection des données."

In [27]:
retrieved_jobs

DetailedJob(id='11adbf10-bc96-4f91-bd95-b7ff41f2b83c', hyperparameters=TrainingParameters(training_steps=50, learning_rate=0.0001), fine_tuned_model=None, model='mistral-small-latest', status='RUNNING', job_type='FT', created_at=1719587199, modified_at=1719587199, training_files=['a1818d5d-1aa9-47ec-af49-4de158e3c7d8'], validation_files=['f73524a6-0533-4f91-8a13-b3bef2a83db2'], object='job', integrations=[], events=[Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719587199), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719587199)], checkpoints=[], estimated_start_time=None)